## Audio Experiment 3

This program replicates the funtionality of the learning server part as described in the polyphonic_track project by Jay Miller, Available at https://github.com/jaym910/polyphonic_track

The program works as follow: when the user click the “Learn Notes” button, the first note will be displayed (in our case E2, corresponding to the lowest possible note on the guitar neck), and the program will continuously scan the live audio. As soon as a signal above a certain threshold is detected, the FFT is calculated and the output is presented in the “Detected Note” graph. If the system detects the right note, then a positive message is displayed and after 1 second, the cycle will repeat for the next note, until the highest note is processed.
Otherwise, if the detected note differs from the expected one, an appropriate message is displayed, and after 1 second the cycle restarts for the same note.

The collected FFTs can then be used to predict chords.

In [3]:
import threading
import time
import pickle

import pyaudio

import numpy as np

import ipywidgets as widgets

from bokeh.io import show, output_notebook, push_notebook
from bokeh.plotting import figure
from bokeh.layouts import row, column

from utilities_globals import *

# Loading BokehJS
output_notebook()

Loading BokehJS ...

In [4]:
def learningMode():
    global min_note_midi
    global currentNoteIndex
    global pitches_per_index
    global data_per_note
    global datafilename

    
    min_note = 'E2'
    max_note = 'A4'
    datafilename = 'fretdata.p'
    
    pitches_per_index = create_notebins(min_note=min_note,
                                        max_note=max_note)
    min_note_midi = note_to_midi(min_note)
    max_note_midi = note_to_midi(max_note)
    data_per_note = [[] for i in pitches_per_index]
    currentNoteIndex = 0
    noteLabel.value = pitches_per_index[currentNoteIndex]

    
# GUI Elements
onOffButton = widgets.ToggleButtons(
    description = 'Status',
    options = ['Stop', 'Start'],
    button_style='info'
)
def onOffButtonChanged(v):
    global RUNNING
    global processAudio
    if v['new'] == 'Start':
        print('Start Audio Processing')
        RUNNING = True
        
        thread = threading.Thread(target=processAudio)
        thread.start()
    else:
        print('Stop Audio Processing')
        RUNNING = False
onOffButton.observe(onOffButtonChanged, names = 'value')

learnNotesButton = widgets.ToggleButton(
    value=False,
    description='Learn Notes',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    icon='check'
)
def onLearnNotesButtonChanged(v):
    global LEARN_NOTE
    global FIND_NEXT_NOTE
    global RUNNING
    
    if RUNNING:
        if v['new']:
            print('Enter Note Learning mode')
            
            learningMode()
            LEARN_NOTE = True    
            FIND_NEXT_NOTE = True
        else:
            print('Exit Note Learning mode')
            LEARN_NOTE = False
            FIND_NEXT_NOTE = False
learnNotesButton.observe(onLearnNotesButtonChanged, names = 'value')

noteLabel = widgets.Text(
    value='Press "Learn notes" to start',
    description='Learning:',
    disabled=False
)

    
CHUNK = 8192 # number of data points to read at a time
RATE = 44100 # time resolution of the recording device (Hz)
LEARN_NOTE = False
FIND_NEXT_NOTE = False
NOTE_FOUND = False
RUNNING = False

def getFFT():
    global data # int16
    # The FFT of the signal
    sig_fft = np.fft.fft(data) # complex128
    # And the power (sig_fft is of complex dtype)
    power = np.abs(sig_fft) # float64

    # The corresponding frequencies
    sample_freq = np.fft.fftfreq(data.size, d=1/RATE) # float64

    return sample_freq, power

    
# Live audio
volumeX = range(CHUNK)
data = np.zeros(CHUNK, dtype=np.int16)
avol = figure(plot_width=450, plot_height = 250, y_range=[-32768 *0.30, 32767 * 0.3], 
              x_axis_label = 'Time', y_axis_label = 'Amplitude',
              title = 'Live Audio')
avol.line(volumeX, data)

# FFT
fft = figure(plot_width=450, plot_height = 250, x_range= [0, 4000], 
              y_range=[0, 2000000], x_axis_label = 'Frequency (Hz)', 
              y_axis_label = 'FFT Magnitude (power)',
              title = 'Live Audio')
fft_freq, fft_power = getFFT()
fft.line(fft_freq, fft_power)

# FFT of currently detected note
n_fft = figure(plot_width=450, plot_height = 250, x_range= [0, 4000], 
              y_range=[0, 2000000], x_axis_label = 'Frequency (Hz)',
              y_axis_label = 'FFT Magnitude (power)',
              title = 'Detected Note')
n_fft_freq = np.copy(fft_freq)
n_fft_power = np.copy(fft_power)
n_fft.line(n_fft_freq, n_fft_power)

# The UI
target = show(column(row(avol, fft), n_fft), notebook_handle = True)
  
# Audio data callback
def audio_callback(in_data, frame_count, time_info, status):
    global LEARN_NOTE
    global FIND_NEXT_NOTE
    global NOTE_FOUND
    
    global data
    global fft_freq
    global fft_power
    global n_fft_freq
    global n_fft_power
    global maxFreq
    
    data = np.frombuffer(in_data, count=frame_count, dtype=np.int16)
    fft_freq, fft_power = getFFT()
    
    if LEARN_NOTE and FIND_NEXT_NOTE:
        maxPower = fft_power.max()
        if maxPower > 1000000:
            FIND_NEXT_NOTE = False
            NOTE_FOUND = True
            n_fft_freq = np.copy(fft_freq)
            n_fft_power = np.copy(fft_power)
            maxFreq = np.abs(n_fft_freq[n_fft_power.argmax()])            
        
    return (data, pyaudio.paContinue)

def findNextNote():
    global FIND_NEXT_NOTE
    global noteLabel
    
    time.sleep(1)
    noteLabel.value = pitches_per_index[currentNoteIndex]
    FIND_NEXT_NOTE = True

def processAudio():     
    global CHUNK
    global RUNNING
    global FIND_NEXT_NOTE
    global NOTE_FOUND

    global avol
    global fft
    global stream
    global data
    global maxFreq
    global data_per_note
    global pitches_per_index
    global currentNoteIndex
    global datafilename
    
    p=pyaudio.PyAudio() # start the PyAudio class
    stream=p.open(format=pyaudio.paInt16,channels=1,rate=RATE,input=True,
              frames_per_buffer=CHUNK, stream_callback=audio_callback) #uses default input device

    while RUNNING:
        # Display info currently in data
        # Display audio volume info
        avol.renderers = []    
        avol.line(volumeX, data)

        # Display fft
        fft.renderers = []
        fft.line(fft_freq, fft_power)
        
        # Display Note
        if NOTE_FOUND:
            NOTE_FOUND = False
            n_fft.renderers = []
            n_fft.line(n_fft_freq, n_fft_power)
            if pitch(maxFreq) == pitches_per_index[currentNoteIndex]:
                data_per_note[currentNoteIndex] = n_fft_freq
                noteLabel.value = pitch(maxFreq) + ' OK'
                currentNoteIndex += 1
                if currentNoteIndex == len(pitches_per_index):
                    print('Saving data')
                    pickle.dump(data_per_note, open(datafilename, "wb"))
                    noteLabel.value = 'Learning complete'
                    learnNotesButton.value = False
                else:
                    time.sleep(1)

                    
            else:
                noteLabel.value = pitches_per_index[currentNoteIndex] + ' - got ' + pitch(maxFreq) + '(' + str(int(maxFreq)) + ')'
                
            if currentNoteIndex < len(pitches_per_index):  
                threading.Thread(target=findNextNote).start()            

        # Update notebook   
        push_notebook(handle=target)        

    # Free resources
    stream.stop_stream()
    stream.close()
    p.terminate()

    
    
    
# Display UI 
display(widgets.VBox([widgets.HBox([onOffButton, learnNotesButton]), noteLabel], justify_content = 'align-items'))

Start Audio Processing
Enter Note Learning mode


NameError: name 'create_notebins' is not defined

Stop Audio Processing
